In [2]:
!pip install transformers
!pip install torch
!pip install accelerate
!pip install numpy
!pip install tensorboardx
!pip install datasets

from transformers import AutoModelForQuestionAnswering, AutoTokenizer, Trainer, TrainingArguments, TrainerCallback
from datasets import load_dataset
import torch
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}") 

# Load the SQuAD dataset
squad = load_dataset("squad")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 63.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 41.4 MB/s eta 0:00:00 0:00:01
device: cuda


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [3]:
# Replace original teacher model initialization
teacher_model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
teacher_model = AutoModelForQuestionAnswering.from_pretrained(
    teacher_model_name,
    trust_remote_code=True  # Required for DeepSeek models
)
teacher_tokenizer = AutoTokenizer.from_pretrained(
    teacher_model_name,
    trust_remote_code=True,
    use_fast=False  # Recommended for DeepSeek models
)

# Modify preprocessing for DeepSeek's tokenization
def preprocess_teacher_train(example):
    inputs = teacher_tokenizer(
        example["question"],
        example["context"],
        truncation=True,
        max_length=512,  # Matches DeepSeek's context window
        stride=96,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
        add_special_tokens=True  # Explicitly enable special tokens
    )
    

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Some weights of Qwen2ForQuestionAnswering were not initialized from the model checkpoint at deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight', 'transformer.embed_tokens.weight', 'transformer.layers.0.input_layernorm.weight', 'transformer.layers.0.mlp.down_proj.weight', 'transformer.layers.0.mlp.gate_proj.weight', 'transformer.layers.0.mlp.up_proj.weight', 'transformer.layers.0.post_attention_layernorm.weight', 'transformer.layers.0.self_attn.k_proj.bias', 'transformer.layers.0.self_attn.k_proj.weight', 'transformer.layers.0.self_attn.o_proj.weight', 'transformer.layers.0.self_attn.q_proj.bias', 'transformer.layers.0.self_attn.q_proj.weight', 'transformer.layers.0.self_attn.v_proj.bias', 'transformer.layers.0.self_attn.v_proj.weight', 'transformer.layers.1.input_layernorm.weight', 'transformer.layers.1.mlp.down_proj.weight'

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [4]:
# Should do a training pass on SQuAD1.1 to initialise the weights in the Teacher model.
# This will also stop the warning from displaying

# New function to preprocess training data for teacher
def preprocess_teacher_train(example):
    # roberta-base-squad-v1
    #inputs = teacher_tokenizer(
    #    example["question"],
    #    example["context"],
    #    truncation=True,
    #    max_length=384,
    #    stride=128,
    #    return_overflowing_tokens=True,
    #    return_offsets_mapping=True,
    #    padding="max_length"
    #)
    
    # DeepSeek-R1-Distill-Qwen-1.5B
    inputs = teacher_tokenizer(
        example["question"],
        example["context"],
        truncation=True,
        max_length=512,
        stride=96,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
        add_special_tokens=True  # Explicitly enable special tokens
    )
    
    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = example["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])
        
        sequence_ids = inputs.sequence_ids(i)

        # Handle empty sequence_ids case
        if not sequence_ids:
            start_positions.append(0)
            end_positions.append(0)
            continue

        # Find context start with boundary checks
        idx = 0
        while idx < len(sequence_ids) and sequence_ids[idx] != 1:
            idx += 1
        context_start = idx if idx < len(sequence_ids) else 0

        # Find context end with boundary checks
        while idx < len(sequence_ids) and sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1 if idx > 0 else 0

        # Handle answer position calculation
        if (context_start >= len(offset) or 
            context_end >= len(offset) or
            offset[context_start][0] > end_char or 
            offset[context_end][1] < start_char):
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Find start position with bounds checking
            idx = context_start
            while idx <= context_end and idx < len(offset) and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(min(idx - 1, len(offset)-1))
            
            # Find end position with bounds checking
            idx = context_end
            while idx >= context_start and idx < len(offset) and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(min(idx + 1, len(offset)-1))
    
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# Teacher Training Arguments
teacher_training_args = TrainingArguments(
    output_dir="./teacher_train",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,  # Increased logging frequency
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    learning_rate=1e-5,
    report_to="tensorboard",
    fp16=True,  # Enable mixed precision training
    dataloader_num_workers=4,
)

# Custom progress callback
class TeacherTrainingProgress(TrainerCallback):
    def on_train_begin(self, args, state, control, **kwargs):
        print(f"🚀 Starting training with {args.num_train_epochs} epochs")
        print(f"📊 Batch size: {args.per_device_train_batch_size}")
        print(f"🔍 Evaluation every {args.eval_steps} steps")

    def on_epoch_begin(self, args, state, control, **kwargs):
        print(f"\n⏳ Starting epoch {state.epoch}/{args.num_train_epochs}")
        
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and 'loss' in logs:
            print(f"Step {state.global_step}: Loss {logs['loss']:.4f}")
        if logs and 'eval_loss' in logs:
            print(f"Validation Loss: {logs['eval_loss']:.4f}")
            print(f"Exact Match: {logs['eval_exact_match']:.2f}%")
            print(f"F1 Score: {logs['eval_f1']:.2f}%")

# Add metrics computation to Trainer
def compute_metrics(p):

    # Convert logits to predictions
    start_pred = np.argmax(p.predictions[0], axis=1)
    end_pred = np.argmax(p.predictions[1], axis=1)
    
    # Get true positions
    start_true = p.label_ids[0]
    end_true = p.label_ids[1]
    
    # Calculate exact match
    exact_matches = np.logical_and(
        start_pred == start_true,
        end_pred == end_true
    )

    # Calculate span F1
    def overlap_f1(p_start, p_end, t_start, t_end):
        pred_span = set(range(p_start, p_end+1))
        true_span = set(range(t_start, t_end+1))
        overlap = len(pred_span & true_span)
        precision = overlap / len(pred_span) if pred_span else 0
        recall = overlap / len(true_span) if true_span else 0
        return 2*(precision*recall)/(precision+recall) if (precision+recall) else 0
    
    f1_scores = [
        overlap_f1(sp, ep, st, et)
        for sp, ep, st, et in zip(start_pred, end_pred, start_true, end_true)
    ]
    
    return {
        "exact_match": np.mean(exact_matches) * 100,
        "f1": np.mean(f1_scores) * 100
    }


# Create Trainer for teacher
teacher_trainer = Trainer(
    model=teacher_model,
    args=teacher_training_args,
    train_dataset=squad["train"].map(preprocess_teacher_train, batched=True, remove_columns=squad["train"].column_names),
    eval_dataset=squad["validation"].map(preprocess_teacher_train, batched=True, remove_columns=squad["validation"].column_names),
    tokenizer=teacher_tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[TeacherTrainingProgress()]
)


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

/tmp/ipykernel_351/3919797043.py:162: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  teacher_trainer = Trainer(


In [5]:
# Train teacher model
print("\nTraining Teacher Model on SQuAD1.1...")
teacher_trainer.train()
teacher_model.save_pretrained("./DeepSeek-R1-Distill-Qwen-1.5B-trained")
teacher_tokenizer.save_pretrained("./DeepSeek-R1-Distill-Qwen-1.5B-trained")


Training Teacher Model on SQuAD1.1...
🚀 Starting training with 3 epochs
📊 Batch size: 16
🔍 Evaluation every None steps

⏳ Starting epoch 0/3


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch,Training Loss,Validation Loss,Exact Match,F1
1,0.000000,nan,0.479819,0.479819
2,0.000000,nan,0.479819,0.479819
3,0.000000,nan,0.479819,0.479819


NaN or Inf found in input tensor.


Step 50: Loss 0.0000


...

NaN or Inf found in input tensor.


Step 16450: Loss 0.0000


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Validation Loss: nan
Exact Match: 0.48%
F1 Score: 0.48%


('./DeepSeek-R1-Distill-Qwen-1.5B-trained/tokenizer_config.json',
 './DeepSeek-R1-Distill-Qwen-1.5B-trained/special_tokens_map.json',
 './DeepSeek-R1-Distill-Qwen-1.5B-trained/tokenizer.json')

In [6]:

print("\nRe-loading optimized teacher model")
teacher_model = AutoModelForQuestionAnswering.from_pretrained(".//DeepSeek-R1-Distill-Qwen-1.5B-trained")
teacher_tokenizer = AutoTokenizer.from_pretrained(".//DeepSeek-R1-Distill-Qwen-1.5B-trained")


Re-loading optimized teacher model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# Define evaluation function
def evaluate_model(model, tokenizer, dataset):
    model.to(device)
    
    metric = evaluate.load("squad")
    predictions = []
    references = []

    for example in tqdm(dataset, desc="Evaluating"):
        # Tokenize inputs
        inputs = tokenizer(
            example["context"], example["question"], truncation=True, padding=True, return_tensors="pt"
        )

        # Move inputs to the same device as the model
        inputs = {key: value.to(device) for key, value in inputs.items()}
        
        # Get model outputs
        outputs = model(**inputs)
        start_logits, end_logits = outputs.start_logits, outputs.end_logits
        start_idx = torch.argmax(start_logits, dim=-1).item()
        end_idx = torch.argmax(end_logits, dim=-1).item()
        
        # Decode prediction
        prediction = tokenizer.decode(inputs["input_ids"][0][start_idx:end_idx + 1])
        
        # Append to predictions
        predictions.append({
            "id": example["id"],
            "prediction_text": prediction
        })

        # Append to references (ground truth)
        references.append({
            "id": example["id"],
            "answers": example["answers"]
        })

    # Compute metrics
    result = metric.compute(predictions=predictions, references=references)
    print(f"Exact Match: {result['exact_match']:.2f}%")
    print(f"F1 Score: {result['f1']:.2f}%\n")
    
    return result
    

In [8]:
def preprocess_validation_data(example):
    # Tokenize context and question
    inputs = student_tokenizer(
        example["context"],
        example["question"],
        truncation=True,
        padding="max_length",
        max_length=384,
    )
    return inputs

In [11]:
# Evaluate teacher model on validation set
import evaluate
from tqdm import tqdm

validation_dataset = squad["validation"]

#if debugging: 
#    validation_set = validation_dataset.select(range(5000))
#    validation_dataset = validation_set

print("Teacher Model Evaluation")
result = evaluate_model(teacher_model, teacher_tokenizer, validation_dataset)

Teacher Model Evaluation


Evaluating: 100%|██████████| 10570/10570 [04:18<00:00, 40.88it/s]


Exact Match: 0.00%
F1 Score: 0.00%

